# --- Day 9: Stream Processing ---

A large stream blocks your path. According to the locals, it's not safe to cross the stream at the moment because it's full of garbage. You look down at the stream; rather than water, you discover that it's a stream of characters.

You sit for a while and record part of the stream (your puzzle input). The characters represent groups - sequences that begin with { and end with }. Within a group, there are zero or more other things, separated by commas: either another group or garbage. Since groups can contain other groups, a } only closes the most-recently-opened unclosed group - that is, they are nestable. Your puzzle input represents a single, large group which itself contains many smaller ones.

Sometimes, instead of a group, you will find garbage. Garbage begins with < and ends with >. Between those angle brackets, almost any character can appear, including { and }. Within garbage, < has no special meaning.

In a futile attempt to clean up the garbage, some program has canceled some of the characters within it using !: inside garbage, any character that comes after ! should be ignored, including <, >, and even another !.

You don't see any characters that deviate from these rules. Outside garbage, you only find well-formed groups, and garbage always terminates according to the rules above.

Here are some self-contained pieces of garbage:

```
<>, empty garbage.
<random characters>, garbage containing random characters.
<<<<>, because the extra < are ignored.
<{!>}>, because the first > is canceled.
<!!>, because the second ! is canceled, allowing the > to terminate the garbage.
<!!!>>, because the second ! and the first > are canceled.
<{o"i!a,<{i<a>, which ends at the first >.
```
  
Here are some examples of whole streams and the number of groups they contain:

```
{}, 1 group.
{{{}}}, 3 groups.
{{},{}}, also 3 groups.
{{{},{},{{}}}}, 6 groups.
{<{},{},{{}}>}, 1 group (which itself contains garbage).
{<a>,<a>,<a>,<a>}, 1 group.
{{<a>},{<a>},{<a>},{<a>}}, 5 groups.
{{<!>},{<!>},{<!>},{<a>}}, 2 groups (since all but the last > are canceled).
```

Your goal is to find the total score for all groups in your input. Each group is assigned a score which is one more than the score of the group that immediately contains it. (The outermost group gets a score of 1.)

```
{}, score of 1.
{{{}}}, score of 1 + 2 + 3 = 6.
{{},{}}, score of 1 + 2 + 2 = 5.
{{{},{},{{}}}}, score of 1 + 2 + 3 + 3 + 3 + 4 = 16.
{<a>,<a>,<a>,<a>}, score of 1.
{{<ab>},{<ab>},{<ab>},{<ab>}}, score of 1 + 2 + 2 + 2 + 2 = 9.
{{<!!>},{<!!>},{<!!>},{<!!>}}, score of 1 + 2 + 2 + 2 + 2 = 9.
{{<a!>},{<a!>},{<a!>},{<ab>}}, score of 1 + 2 = 3.
```
   
What is the total score for all groups in your input?

First, I'm going to remove the garbage characters by:
- find the location of `<`
- find the location of `<`. This should not be right after `!`

In [48]:
def remove_garbage(stream):
    """takes in a stream of characters and returns a cleaned up version
    garbage is characters b/w < and > and ! inside garbage cancels the character after it"""
    
    clean_input = ""
    inside_garbage = False
    escape_next = False
    
    for i, char in enumerate(stream):
        
        if not inside_garbage:
            if char == "<":
                inside_garbage = True
            else:
                clean_input += char
        else: # we are inside garbage now
            if char == "!" and not escape_next:
                escape_next = True
            elif char == ">" and not escape_next:
                inside_garbage = False
            elif char is not "!" and escape_next:
                escape_next = False
            elif char is "!" and escape_next:
                escape_next = False
        
    return clean_input

print(remove_garbage("<{!>}>"))
print(remove_garbage("{{<a!>},{<a!>},{<a!>},{<ab>}}"))
print(remove_garbage("{}"))
print(remove_garbage("{{<!!>},{<!!>},{<!!>},{<!!>}}"))
print(remove_garbage("{{<a!>},{<a!>},{<a!>},{<ab>}}"))


{{}}
{}
{{},{},{},{}}
{{}}


This looks like a job for recursion, but I'm going to do it the easy way:

In [68]:
def score(stream):
    """takes in a stream of characters and returns the score"""
    stream = remove_garbage(stream)
    
    inside_group = False
    group_count = 0
    total_count = 0
    
    for i, char in enumerate(stream):
        if char == "{":
            inside_group = True
            group_count += 1
        if char == "}":
            total_count += group_count
            group_count -= 1
    
    return total_count
                
score("{}")   

1

test cases from the examples above:

In [70]:
assert score("{}") == 1
assert score("{{{}}}") == 6
assert score("{{},{}}") == 5
assert score("{{{},{},{{}}}}") == 16
assert score("{<a>,<a>,<a>,<a>}") == 1
assert score("{{<ab>},{<ab>},{<ab>},{<ab>}}") == 9
assert score("{{<!!>},{<!!>},{<!!>},{<!!>}}") == 9
assert score("{{<a!>},{<a!>},{<a!>},{<ab>}}") == 3

In [71]:
with open('puzzle_inputs/day9_input.txt') as f:
    puzzle_input = f.read()
puzzle_input[:700]

'{{{{{<!>,<!o,!!!>{>,{<<a!>u!>eio!>ui>}},{},{{<!!e!>>},{{<uo\'<!!!>>},{<!>\'!>},<e!>,<"ea!!!>!>,<>}}}},{{{{<"a">,<!><<>}}}},{{{{{},{<{!!"!><ae<},!>!>!!!>}\'!!>}},<!!a{!!!!!>!>,<!!!>,<{oiu<a!!,<>},{<>}}}},{{{{<eoa!!eeea{!,<>}},{{<!!!!,}!>!!a!!!}e!>},<!!!!!>!>i!e!>,<!u<!!{!!!>!!!>>}}}},{{{{{<!ea,uio!}"!>},<!>},<u<!!!!o!<,!>>,<!>},<">},{}},{<ua"!>,<!!!>eouaa"o}!!!>!>,<>}},{<a}!>!!<\'a!!<!u!!!!!>!!!>!!!a!>},<!>,<>}},{{{},{{{},<ua}!>!!!>i}}u!!!>},<!>!ii!!\'!,>},{{<{>},{<i<!!!e}!>,<!"\'a!!u!!{\',>}}}},{<!>},<!!!!},o}!!}{!!!">}}},{{{{},{<!>},<!!!!!>}}{\'>}}},{{{{<},{a!>,<>}},{<>,<!i>}},{{{<"u!i{!i}a!!!>"\'>}}}},{{{<<e!\'!!i\'!!}{!>,<u,""!!i<ui!!!!!>!!<>},<!!!>\'u!!ea!>u\'!>}!!!!!>>},{{}},{<"i}u,,>}}}},{{{{{{<\',!'

In [72]:
score(puzzle_input)

12803

# --- Part Two ---

Now, you're ready to remove the garbage.

To prove you've removed it, you need to count all of the characters within the garbage. The leading and trailing < and > don't count, nor do any canceled characters or the ! doing the canceling.

```
<>, 0 characters.
<random characters>, 17 characters.
<<<<>, 3 characters.
<{!>}>, 2 characters.
<!!>, 0 characters.
<!!!>>, 0 characters.
<{o"i!a,<{i<a>, 10 characters.
```

How many non-canceled characters are within the garbage in your puzzle input?

easy, just modify the remove garbage function to count the characters removed 

In [73]:
def remove_garbage_and_count(stream):
    """takes in a stream of characters and returns a cleaned up version
    garbage is characters b/w < and > and ! inside garbage cancels the character after it"""
    
    clean_input = ""
    inside_garbage = False
    escape_next = False
    garbage_count = 0
    
    for i, char in enumerate(stream):
        
        if not inside_garbage:
            if char == "<":
                inside_garbage = True
            else:
                clean_input += char
        else: # we are inside garbage now
            if char == "!" and not escape_next:
                escape_next = True
            elif char == ">" and not escape_next:
                inside_garbage = False
            elif char is not "!" and escape_next:
                escape_next = False
            elif char is "!" and escape_next:
                escape_next = False
            else:
                garbage_count += 1
        
    return garbage_count

print(remove_garbage_and_count("<{!>}>"), "should be 2")
print(remove_garbage_and_count(r'<{o"i!a,<{i<a>'), "should be 10")

2 should be 2
10 should be 10


In [74]:
remove_garbage_and_count(puzzle_input)

6425

## Notes:

- could have used regex to find groups
- my group counting works only for this instance, better solutions exist, look up